In [ ]:
# for first run on vast.ai
%pip install transformer_lens matplotlib openai python-dotenv

In [ ]:
import torch
from transformer_lens import HookedTransformer
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, Union, List
from openai import OpenAI, RateLimitError
from tqdm import tqdm
from dotenv import load_dotenv
import time
import os
from datasets import load_dataset
import json
import random

load_dotenv()

## OpenAI client & helpers

In [ ]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

def ask_yes_no(prompt: str) -> bool:
    done = False
    backoff = 4
    while not done:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that answers yes or no questions. Respond with 'yes' or 'no' only."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                max_tokens=3,
                temperature=0,
            )
            return "yes" in response.choices[0].message.content.lower()
        except RateLimitError as e:
            print(f"Rate limit error: {e}")
            time.sleep(backoff)
            backoff *= 2
            if backoff > 128:
                raise e

def batch_yes_no(batch: List[str], question_fn, progress=True, num_workers=20) -> List[bool]:
    from concurrent.futures import ThreadPoolExecutor
    
    # Process in parallel batches of num_workers
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(question_fn, text) for text in batch]
        results = list(tqdm(
            (f.result() for f in futures),
            total=len(batch),
            desc="Evaluating batch", 
            disable=not progress
        ))
    
    return results

def related(big_batch: List[str], topic: str, progress=True, num_workers=20) -> List[bool]:
    question_fn = lambda text: ask_yes_no(f"Is this text related to {topic}? Text:\n\n{text}")
    return batch_yes_no(big_batch, question_fn, progress, num_workers)

def coherent(batch: List[str], progress=True, num_workers=20) -> List[bool]:
    question_fn = lambda text: ask_yes_no(f"Is this text extremely incoherent? Text:\n\n{text}")
    return batch_yes_no(batch, question_fn, progress, num_workers)

related(["I like the mona lisa", "Walruses are fascinating creatures"], "art")

In [ ]:
inputs = ["I got stuck in traffic (I live in Sicily) on the way to the theater (at a military base) to see Superman Returns, was ",
          "I don't know if this type of movie was as clich\u00e9 then as it seems to be now.<br /><br />Considering how many \"Bad News",
          "Forest of the Damned starts out as five young friends, brother & sister Emilio (Richard Cambridge) & Ally (Sophie Holland) along with J",
          "I saw this DVD on sale and bought it without a second thought, despite not even having known it was out since this is one of my favorite books of",
          "I don't know much about Tobe Hooper, or why he gets his name in the title, but maybe he shouldn't have bothered. As another",
          "I liked the understated character that Laura Linney played in 'Love Actually', and she is very good in 'Man of the Year'.<br /><",
          "This film is probably the worst movie I have watched in a very long time. The acting is so wooden a door could have done a better job. The",
          "This is the kind of film one watches in gape-jawed, horrified silence, and yet continues to watch, mesmerized, as if watching a",
          "Sorry I couldn't disagree more,with the last comments. frankly I thought this was worse than Carry on Columbus, enough said. Last film for THE usually",
          "Next to the slasher films of the 1970s and 80s, ones about the walking dead were probably the second most popular horror sub-gen", "This is the first Guinea Pig film from Japan and this is the sickest, in my opinion. A bunch of guys torture a girl for several days before", "\"True\" story of a late monster that appears when an American industrial plant begins polluting the waters. Amusing, though not really good, monster film", "Even if you could get past the idea that these boring characters personally witnessed every Significant Moment of the 1960s (ok, so Katie didn't join", "I saw the MST3K version of \"Deathstalker III\" and loved the movie so much -- even \"unmystied\" -- that I", "I have a question for the writers and producers of \"Prozac Nation\": What is the root cause and what is the solution to the widespread problem of personal"]

results = coherent(inputs)

for i, r in enumerate(results):
    print(f"{inputs[i]}\n{r}\n")

In [ ]:
torch.set_grad_enabled(False)  # save memory
model_name = "meta-llama/Meta-Llama-3-8B"
# model_name = "gpt2-xl"
device = "cuda:0"
model = HookedTransformer.from_pretrained(model_name, device=device)
model.eval()
if torch.cuda.is_available():
  model.to(device)

In [3]:
SEED = 0
sampling_kwargs = dict(temperature=1.0, top_p=0.3, freq_penalty=1.0)

In [7]:
tlen = lambda prompt: model.to_tokens(prompt).shape[1]
pad_right = lambda prompt, length: prompt + " " * (length - tlen(prompt))

def pad_both(p_add, p_sub):
  l = max(tlen(p_add), tlen(p_sub))
  return pad_right(p_add, l), pad_right(p_sub, l)

In [5]:
def get_resid_pre(prompt: str, layer: int):
    name = f"blocks.{layer}.hook_resid_pre"
    cache, caching_hooks, _ = model.get_caching_hooks(lambda n: n == name)
    with model.hooks(fwd_hooks=caching_hooks):
        _ = model(prompt)
    return cache[name]


def get_act_diff(prompt_add: str, prompt_sub: str, layer: int):
    act_add = get_resid_pre(prompt_add, layer)
    act_sub = get_resid_pre(prompt_sub, layer)
    return act_add - act_sub # if this errors you forgot to pad

In [14]:
from functools import partial

def ave_hook(act_diff, resid_pre, hook):
    global h
    h = hook
    if resid_pre.shape[1] == 1:
        return  # caching in model.generate for new tokens

    # We only add to the prompt (first call), not the generated tokens.
    ppos, apos = resid_pre.shape[1], act_diff.shape[1]
    assert apos <= ppos, f"More mod tokens ({apos}) then prompt tokens ({ppos})!"

    # add to the beginning (position-wise) of the activations
    resid_pre[:, :apos, :] += act_diff


from typing import Union, List
def hooked_generate(prompt_batch: Union[List[str], torch.Tensor], fwd_hooks=[], seed=None, verbose=False, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        if isinstance(prompt_batch, list):
            tokenized = model.to_tokens(prompt_batch)
        else:
            tokenized = prompt_batch
        r = model.generate(input=tokenized, do_sample=True, verbose=verbose, **kwargs)
    return r


def generate_hooked(prompt_batch: List[str], prompt_add: str, prompt_sub: str, act_name: int, coeff: float, verbose=False, max_new_tokens=50):
  prompt_add, prompt_sub = pad_both(prompt_add, prompt_sub)
  act_diff = coeff*get_act_diff(prompt_add, prompt_sub, act_name)
  editing_hooks = [(f"blocks.{act_name}.hook_resid_pre", partial(ave_hook, act_diff))]
  hooked_res = hooked_generate(prompt_batch, editing_hooks, seed=SEED, verbose=verbose, **sampling_kwargs, max_new_tokens=max_new_tokens)
  return hooked_res


def generate_both(prompt_batch: List[str], prompt_add: str, prompt_sub: str, act_name: int, coeff: float):
  hooked_res = generate_hooked(prompt_batch, prompt_add, prompt_sub, act_name, coeff)
  vanilla_res = generate_hooked(prompt_batch, prompt_add, prompt_sub, act_name, 0.0)
  return hooked_res, vanilla_res

In [ ]:
# Specific to the love/hate example
# commented out bc wastes precious memory demonstrating stuff that works
if 1:
    prompt_add, prompt_sub = "science", ""
    coeff = 4.0
    act_name = 6
    prompt = "Did you know that"

    print("Generating...")
    hooked_res, vanilla_res = generate_both([prompt]*20, prompt_add, prompt_sub, act_name, coeff)
    print("Done generating!")

    # Print results, removing the ugly beginning of sequence token
    # NOTE: Has endoftext a bunch too
    hooked_strs = model.to_string(hooked_res[:, 1:])
    vanilla_strs = model.to_string(vanilla_res[:, 1:])

    hooked_rel = related(hooked_strs, prompt_add)
    vanilla_rel = related(vanilla_strs, prompt_add)
    print()
    print(f"Hooked: {np.mean(hooked_rel)}")
    print(f"Vanilla: {np.mean(vanilla_rel)}")

In [ ]:
torch.random.manual_seed(SEED)
tokenized = model.to_tokens([prompt]*20)
unhooked_res = model.generate(tokenized, do_sample=True, max_new_tokens=50, **sampling_kwargs)
unhooked_strs = model.to_string(unhooked_res[:, 1:])
unhooked_rel = related(unhooked_strs, prompt_add)
print(f"Unhooked: {np.mean(unhooked_rel)}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
hf_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")
hf_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

hf_model.to("cuda:0")

In [48]:
# perplexity code adapted from https://github.com/eth-sri/language-model-arithmetic/blob/6779b40702d5214525a21e51c91d5042f873bd81/src/model_arithmetic/evaluation.py#L285

def my_perplexity(model, prompts: List[str], generated: List[str]):
    prompts_tokenized = model.to_tokens(prompts)
    generated_tokenized = model.to_tokens(generated)

    output = model.forward(generated_tokenized, return_type="loss", loss_per_token=True)

    # Create attention mask that excludes EOS tokens
    attention_mask = (results != model.tokenizer.eos_token_id)

    # output.shape is 1 less than results.shape, exclude last token (we cant compute loss for the last token)
    attention_mask = attention_mask[:, :-1]

    # exclude prompt from perplexity calculation
    output = output[:, prompts_tokenized.shape[1]:]
    attention_mask = attention_mask[:, prompts_tokenized.shape[1]:]

    # compute perplexities :)
    perplexities = ((attention_mask * output).sum(-1) / attention_mask.sum(-1)).exp().tolist()
    return perplexities



# same function but do the forward pass for every input at once
def compute_perplexities_batched(dataset, model, tokenizer, batch_size=8, **kwargs):
    perplexities = []
    sum_nllos = 0
    n_tokens = 0
    
    for i in range(0, len(dataset), batch_size):
        batch = dataset.iloc[i:i+batch_size]
        
        # Get input lengths for each sample in batch
        input_lengths = []
        combined_sentences = []
        for _, sample in batch.iterrows():
            if len(sample['generated']) == 0:
                input_lengths.append(0)
                continue
            input_encodings = tokenizer(sample['input'], return_tensors='pt')
            input_lengths.append(input_encodings['input_ids'].size(1))
            combined_sentences.append(sample['input'] + sample['generated'])
            
        if not combined_sentences:
            continue
            
        # Tokenize full batch
        encodings = tokenizer(combined_sentences, return_tensors='pt', padding=True)
        input_ids = encodings['input_ids'].to(model.device)
        attention_mask = encodings['attention_mask'].to(model.device)
        
        with torch.no_grad():
            # logits = model(input_ids, labels=input_ids, attention_mask=attention_mask).logits # hf_model
            logits = model(input_ids, attention_mask=attention_mask) # transformerlens model
            logprobs = logits.log_softmax(dim=-1)
        
        loss_func = torch.nn.NLLLoss(ignore_index=-100, reduction='sum')
        
        # Calculate loss for each sample in batch
        for j in range(len(combined_sentences)):
            input_length = input_lengths[j]
            if input_length == 0:
                perplexities.append(None)
                continue
                
            loss = loss_func(
                logprobs[j, input_length:-1, :].contiguous(),
                input_ids[j, input_length+1:].contiguous()
            )
            loss = loss.to(torch.float32).detach().cpu().numpy()
            
            n_tokens_here = (attention_mask[j, input_length+1:] == 1).sum().item()
            
            if n_tokens_here > 0:
                perplexity = np.exp(loss / n_tokens_here)
                sum_nllos += loss
                n_tokens += n_tokens_here
                if not np.isnan(perplexity):
                    perplexities.append(perplexity)
                else:
                    perplexities.append(None)
            else:
                perplexities.append(None)
                
    return perplexities, sum_nllos, n_tokens


def get_perplexity(dataset, model, **kwargs):
    """
    Calculates the perplexity of the generated sentences.

    Args:
        dataset (pd.DataFrame): The dataset to be used for evaluation. Has columns "input" (for input text), "generated" (for generated text). 
        model (PreTrainedModel): The model to be evaluated.
        tokenizer (Tokenizer): The tokenizer to be used for tokenizing the sentences.
        **kwargs: Additional keyword arguments.
    """
    perplexities, sum_nllos, n_tokens = compute_perplexities_batched(dataset, model, model.tokenizer, **kwargs)

    average = np.mean(perplexities)
    median = np.median(perplexities)
    real = np.exp(sum_nllos / n_tokens)

    return {
        "average": average,
        "median": median,
        "correct_perplexity": real
    }


import pandas as pd

# NOTE: THIS IS WRONG (we passed full completions instead of just generations),
# But that's STILL FINE because we're just making sure my batched version WORKS THE SAME.
df = pd.DataFrame({"input": [prompt]*20, "generated": hooked_strs})
hf_tokenizer.pad_token = model.tokenizer.pad_token
model.device = next(model.parameters()).device
r = get_perplexity(df, model)

# gold standard (their code unmodified with hf model loaded separately)
gold = {'average': 2.9299599115344255, 'median': 2.7279913006370267, 'correct_perplexity': 2.8266689020402422}

# compute diff
diff = {k: r[k] - gold[k] for k in gold}
diff

{'average': 2.634668080148117e-06,
 'median': 4.75194498417153e-06,
 'correct_perplexity': 2.8177933466544403e-06}

In [ ]:
# gold standards (hf_model, before port to transformerlens)
# 1. hooked_strs:
{'average': 2.9299599115344255,
 'median': 2.7279913006370267,
 'correct_perplexity': 2.8266689020402422}

# 2. vanilla_strs & unhooked_strs (same)
{'average': 2.9692005455398056,
 'median': 2.885386634383141,
 'correct_perplexity': 2.8907230675863858}

In [35]:
perplexities_per_sample = my_perplexity(model, hooked_res)
{
    'average': np.mean(perplexities_per_sample),
    'median': np.median(perplexities_per_sample),
}

{'average': 3.307980924844742, 'median': 3.279552698135376}

In [ ]:
# hooked_res is coeff=4.0 hooked
# vanilla_res is coeff=0 hooked
# unhooked_res is pure model.generate no hooks
for results in [hooked_res, vanilla_res, unhooked_res]:
    output = model.forward(results, return_type="loss", loss_per_token=True)

    # Create attention mask that excludes EOS tokens
    attention_mask = (results != model.tokenizer.eos_token_id)

    # output.shape is 1 less than results.shape, exclude last token (we cant compute loss for the last token)
    attention_mask = attention_mask[:, :-1]

    # compute perplexities :)
    perplexities = ((attention_mask * output).sum(-1) / attention_mask.sum(-1)).exp().tolist()
    print(perplexities[:10])
    print("-"*80)

In [ ]:
# filter imdb prompts to remove all things that already talk about one of the topics
TOPICS = ["art", "finance", "music", "politics", "science", "weddings"]

try:
  #1/0 # uncomment to regenerate cache
  imdb_prompts = json.load(open('imdb_prompts_filtered.json'))
except Exception as e:
  ds = load_dataset("stanfordnlp/imdb")

  imdb_sample = random.sample(ds['test']['text'], 3000)
  imdb_prompts = [x[:256] for x in imdb_sample]
  imdb_prompts = [" ".join(x.split(" ")[:-1]) for x in imdb_prompts]
  imdb_prompts[:10]

  for topic in TOPICS:
    rel = related(imdb_prompts, topic, num_workers=100, progress=True)
    imdb_prompts = [x for x, y in zip(imdb_prompts, rel) if not y]
    print(f"{topic}: removed {sum(rel)} ({np.mean(rel)*100:.1f}%) new len prompts: {len(imdb_prompts)}")
    open('imdb_prompts_filtered.json', 'w').write(json.dumps(imdb_prompts))

imdb_prompts[:10]

In [ ]:
# Configuration
COEFFICIENTS = [0.0, 0.5, 1.0, 2.0]
SEED = 0
# PROMPT_BATCH = ["Did you know that"] * 100 # old paper thing
# char level instead of token level
# PROMPT_BATCH = [" ".join(x[:100].split(" ")[:-1]) for x in imdb_prompts[:100]]

# use model.to_tokens to truncate each to 32 tokens
PROMPT_BATCH = [model.to_string(model.to_tokens(x)[0][:32].tolist()[1:]) for x in tqdm(imdb_prompts[:1000])]

act_name = 6

In [33]:
def generate_big_hooked_batch(prompt_batch: List[str], padd: str, psub: str, act_name: int, coeff: float):
  "Generate in cases where processing prompt_batch all at once would make us run out of memory."

  batch_size = 60 # XXX: REDUCE THIS IF UR NOT ON A h100

  res_strs = []
  disable_progress = len(prompt_batch) < batch_size
  for i in tqdm(range(0, len(prompt_batch), batch_size), desc="Prompt batch", disable=disable_progress):
    batch = prompt_batch[i:i+batch_size]
    hooked_res = generate_hooked(batch, padd, psub, act_name, coeff)
    res_strs += model.to_string(hooked_res[:, 1:])
  return [x.replace("<|endoftext|>", "") for x in res_strs]

## Topic steering

Re-does the topic steering experiment. Very customizable.

In [ ]:
from threading import Thread

completions = {}
results = {}

PROMPT_TEMPLATE = "I talk about {topic} constantly"
NEG_PROMPT_TEMPLATE = "I do not talk about {topic} constantly"

# Initialize results
for topic in TOPICS:
    results[topic] = []

threads = []

# Generate completions and spawn related tasks
for topic in tqdm(TOPICS, desc="topics"):
    prompt_add, prompt_sub = pad_both(PROMPT_TEMPLATE.format(topic=topic), 
                                    NEG_PROMPT_TEMPLATE.format(topic=topic))
    completions[topic] = {}
    
    for coeff in COEFFICIENTS:
        hooked_strs = generate_big_hooked_batch(PROMPT_BATCH, prompt_add, prompt_sub, 
                                              act_name, coeff)
        completions[topic][coeff] = hooked_strs
        
        def run_related(t=topic, c=coeff, h=hooked_strs):
            rel = related(h, t, num_workers=100, progress=False)
            score = np.mean(rel)
            results[t].append(score)
            print(f"topic {t} coeff {c} -- rel: {score}")
            
        thread = Thread(target=run_related)
        thread.start()
        threads.append(thread)

# Wait for all threads
print("Waiting for threads to finish...")
for thread in threads:
    thread.join()

In [22]:
# save full results

import time

fname = f"results/{int(time.time())}.json"
with open(fname, "w") as f:
    json.dump({
        "results": results,
        "completions": completions,
        "meta": {
            "coeffs": COEFFICIENTS,
            "topics": TOPICS,
            "act_name": act_name,
            "seed": SEED,
            "prompt_template": PROMPT_TEMPLATE,
            "neg_prompt_template": NEG_PROMPT_TEMPLATE,
        },
        "prompts": PROMPT_BATCH
    }, f)

print(f"Saved results to {fname}")

In [ ]:
# Load results in case we want to plot a different one
# TODO(easy): Past easy experiment graph explorer widget
if 0:
    fname = 'results/1732323963.json'
    with open(fname) as f:
        file_data = json.load(f)
    results = file_data['results']
    completions = file_data['completions']
    COEFFICIENTS = file_data['meta']['coeffs']
    TOPICS = file_data['meta']['topics']
    act_name = file_data['meta']['act_name']
    SEED = file_data['meta']['seed']
    PROMPT_TEMPLATE = file_data['meta']['prompt_template']
    NEG_PROMPT_TEMPLATE = file_data['meta']['neg_prompt_template']
    PROMPT_BATCH = file_data['prompts']

    print("Loaded results from", fname, "with", len(PROMPT_BATCH), "prompts", "and coeffs", COEFFICIENTS, "and prompt templates:\n", PROMPT_TEMPLATE, "\nand neg prompt templates:\n", NEG_PROMPT_TEMPLATE)


In [ ]:
# Create the topic steering plot WITH baseline included
plt.figure(figsize=(12, 6))
x = np.arange(len(TOPICS))
width = 0.2

subtract = True
for i, coeff in enumerate(COEFFICIENTS):
    relevance_scores = [results[topic][i]*100 for topic in TOPICS]
    plt.bar(x + i * width, relevance_scores, width, label=f'c={coeff}')

plt.xlabel('Topic')
plt.ylabel('% evaluated as relevant')
plt.title('gpt-4o-mini scored relevance of ActAdd completions on generic topics')
plt.xticks(x + width * (len(COEFFICIENTS) - 1) / 2, TOPICS)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Create the visualization WITHOUT baseline included (show diffs)
plt.figure(figsize=(12, 6))
x = np.arange(len(TOPICS))
width = 0.2

subtract = True
for i, coeff in enumerate(COEFFICIENTS[1:]):
    relevance_scores = [results[topic][i]*100 for topic in TOPICS]
    plt.bar(x + i * width, relevance_scores, width, label=f'c={coeff}')

plt.xlabel('Topic')
# plt.ylabel('% evaluated as relevant minus no-editing baseline')
plt.ylabel('Δ% Topic Relevance (vs Unedited)')
plt.title('gpt-4o-mini scored relevance of ActAdd completions on generic topics')
plt.xticks(x + width * (len(COEFFICIENTS[1:]) - 1) / 2, TOPICS)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# works better for long 32 tok imdb prompts:
# prompt_add, prompt_sub = pad_both("I talk about weddings constantly", "I do not talk about weddings constantly")
# coeff = 4.0

# XXX: paper used https://arxiv.org/pdf/2308.10248v3
# XXX: they also used ‘I went up to my friend and said’ as prompt, but
# XXX: imdb prompts are long so this works worse unless we truncate
# XXX: imdb prompts a bunch (what I've done here, to 16 chars.)
prompt_add, prompt_sub = pad_both("weddings", "")
coeff = 1.0

layer_completions = []
# layers = [2, 6, 16, 35]
layers = list(range(0, 48))
pbatch = [p[:16] for p in PROMPT_BATCH] # XXX: Cheating? See comment above
for layer in tqdm(layers + [-1]):
    if layer != -1:
      hooked_res = generate_hooked(pbatch, prompt_add, prompt_sub, layer, coeff) 
    else:
      hooked_res = generate_hooked(pbatch, prompt_add, prompt_sub, 0, 0) # baseline

    layer_completions.append([
        x.replace("<|endoftext|>", "") for x in model.to_string(hooked_res[:, 1:])
    ])
  
baseline_completion = layer_completions.pop()

In [ ]:
# save
fname = f"results/{int(time.time())}-weddings.json"
with open(fname, "w") as f:
    json.dump({
        "layer_completions": layer_completions,
        "layers": layers,
        "baseline_completion": baseline_completion,
        "prompt_batch": PROMPT_BATCH,
        "meta": {
            "coeff": coeff,
            "prompt_add": prompt_add,
            "prompt_sub": prompt_sub,
            "seed": SEED,
            "act_name": act_name,
        }
    }, f)
    print(f"wrote {fname}")

In [ ]:
# Compute wedding-related completions % (figure 3)
wedding_words = (
    "wedding", "weddings", "wed", "marry", "married",
    "marriage", "bride", "groom", "honeymoon",
)

def contains_wedding_words(text: str) -> bool:
    assert isinstance(text, str), f"what the fuck is a '{type(text)}'"
    assert len(text) > 1, f"bruh wtf just '{text}' is all i get"
    return any(word in wedding_words for word in text.lower().split())

wedding_related_pct = [
    np.mean([contains_wedding_words(x) for x in layer_completions])
    for layer_completions in layer_completions
]
baseline_pct = np.mean([contains_wedding_words(x) for x in baseline_completion])

# Create the visualization
plt.figure(figsize=(12, 6))
plt.plot(layers, wedding_related_pct, 'b-', linewidth=1)
plt.axhline(y=baseline_pct, color='k', linestyle='--', alpha=0.3, label='baseline')

plt.xlabel('Layer')
plt.ylabel('% of completions containing wedding words')
plt.grid(True, alpha=0.2)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Graph mean wedding word count per layer (figure 7)
def wedding_word_count(text: str) -> int:
    return sum(word in wedding_words for word in text.lower().split())


wedding_word_counts = [
    np.mean([wedding_word_count(x) for x in lc])
    for lc in layer_completions
]
baseline_count = np.mean([wedding_word_count(x) for x in baseline_completion])

# Create the visualization
plt.figure(figsize=(12, 6))
plt.plot(layers, wedding_word_counts, 'b-', linewidth=1)
plt.axhline(y=baseline_count, color='k', linestyle='--', alpha=0.3, label='baseline')

plt.xlabel('Layer')
plt.ylabel('Mean wedding word count')
plt.grid(True, alpha=0.2)

plt.legend()
plt.tight_layout()
plt.show()

# Grid search



In [8]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import itertools
import json
import hashlib

In [ ]:
PROMPT_BATCHES = {
  "positive": pd.read_csv("prompts/sentiment_imdb_POSITIVE_gridsearch_subset (1).csv")["input"].tolist()[:60],
  "negative": pd.read_csv("prompts/sentiment_imdb_NEGATIVE_gridsearch_subset (1).csv")["input"].tolist()[:60],
  "toxicity": pd.read_csv("prompts/toxicity_pol_gridsearch_subset (1).csv")["input"].tolist()[:60],
}
SELECTION = "toxicity"

PROMPT_BATCH = PROMPT_BATCHES[SELECTION]

prompt_add, prompt_sub = {
  "negative": ("Love", "Hate"),
  "positive": ("Hate", "Love"), # padding reasons: " " is prob better
  "toxicity": ("Love", "Hate"),
}[SELECTION]

max_new_tokens = 32 if SELECTION == "toxicity" else 64

sampling_kwargs = dict(temperature=1.0, top_p=1.0, freq_penalty=1.0) # for llama, new ones
SEED = 0

grid_to_sweep = list(itertools.product(range(0, 23, 3), [-3, -1, 0, 1, 3, 6, 12, 18]))
# 32x32 overkill as fuck. should take around 3hrs for each dataset.
# grid_to_sweep = list(itertools.product(
#     range(0, 32),
#     range(-9, 23),
# ))

exp_name = f"{SELECTION}_n{len(grid_to_sweep)}"
exp_name += ("_fp" if "freq_penalty" in sampling_kwargs else "")
exp_folder = f"grid_results/{exp_name}"
os.makedirs(exp_folder, exist_ok=True)
with open(f"{exp_folder}/meta.json", "w") as f:
  json.dump({
    "prompt_add": prompt_add,
    "prompt_sub": prompt_sub,
    "max_new_tokens": max_new_tokens,
    "sampling_kwargs": sampling_kwargs,
    "PROMPT_BATCH": PROMPT_BATCH,
  }, f)
  print(f"wrote {f.name}")

for i, (l, c) in tqdm(enumerate(grid_to_sweep), total=len(grid_to_sweep)):
  results = generate_hooked(PROMPT_BATCH, prompt_add, prompt_sub, l, c, verbose=False, max_new_tokens=max_new_tokens)
  results_str = model.to_string(results[:, 1:])

  output = model.forward(results, return_type="loss", loss_per_token=True)

  # Create attention mask that excludes EOS tokens
  attention_mask = (results != model.tokenizer.eos_token_id)

  # output.shape is 1 less than results.shape, exclude last token (we cant compute loss for the last token)
  attention_mask = attention_mask[:, :-1]

  # compute perplexities :)
  perplexities = ((attention_mask * output).sum(-1) / attention_mask.sum(-1)).exp().tolist()

  # and json cuz idk why not
  with open(f"{exp_folder}/{i}_l{l}_c{c}.json", "w") as f:
    json.dump({
      "prompt_batch": PROMPT_BATCH,
      "completions": results_str,
      "perplexities": perplexities,
      "layer": l,
      "coeff": c,
      # TODO: More metrics (sentiment, toxicity, % coherent, ?)
    }, f)

In [ ]:
# Read all the json files and plot them
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_heatmap(exp_folder: str, SELECTION: str, values: str = "perplexity", annot: bool = False):
    files = os.listdir(exp_folder)
    data = []

    for f in files:
        if f == "meta.json":
            continue

        with open(exp_folder + "/" + f) as fp:
            d = json.load(fp)
            data.append({
                "layer": d["layer"],
                "coeff": d["coeff"], 
                "perplexity": np.mean(d["perplexities"]), # avg over batch
                "sentiment": np.mean(d["sentiment"]),
            })

    df = pd.DataFrame(data)
    pivot = df.pivot(index="layer", columns="coeff", values=values)

    plt.figure(figsize=(20,10))
    sns.heatmap(pivot, cmap="viridis", annot=annot, fmt='.1f')

    dataset_name = f"IMDB {SELECTION.title()} Reviews" if SELECTION != "toxicity" else "RealToxicityPrompts"
    plt.title(f"{dataset_name}, Completion {values.title()} Heatmap") 

    plt.ylabel("Layer")
    plt.xlabel("Coefficient")
    plt.savefig(f"figures/{exp_name}_{values}_{'noannot_' if not annot else ''}heatmap.png", dpi=300)
    plt.show()

for annot in [False, True]:
    plot_heatmap(exp_folder, SELECTION, "perplexity", annot)
    plot_heatmap(exp_folder, SELECTION, "sentiment", annot)

In [ ]:
# Plot all (for when plotting code updates)

for exp_folder in os.listdir("grid_results"):
  selected_dataset = exp_folder.split("_")[0]
  plot_heatmap(f"grid_results/{exp_folder}", selected_dataset, "perplexity")
  plot_heatmap(f"grid_results/{exp_folder}", selected_dataset, "sentiment")
